# Chapter 07. CNN

## 1. 기존의 신경망과의 비교
우리가 여태까지 봐왔던 인공 신경망은 다음고 같이 **완전연결 계층(=Affine 계층)**으로 이어져 있다. 완전연결 계층이라함은 인접하는 계층의 모든 뉴런과 결합되어 있는 신경망을 말한다. 즉, **입력데이터와 가중치가 1:1 대응하는 것**을 말한다.

<img src="./images/nn.png" width=70%>

----

합성곱 신경망(CNN, Convolutional Neural Network)은 **합성곱 계층(Conv)**와 **풀링 계층(Pooling)**이 추가된 신경망을 말한다. Pooling은 생략될 수 있다.
* `Conv` - `ReLU` - `Pooling`을 반복
* `Affine` - `ReLU`
* `Affine` - `Softmax`

<img src="./images/conv_nn.png" width=70%>

## 2. 완전연결 신경망의 단점
완전연결 신경망의 가장 큰 단점은 **원래 데이터 자체의 형태가 무시**가 된다는 것이다.
* 이미지 데이터는 `세로*가로*채널(색상)`으로 이루어진 3차원 데이터인데 이를 **1차원 데이터로 평탄화(flatten)**시킨다.
* 이 3차원 데이터에는 **이미지 자체의 특징**이 담겨 있다.
    * `ex1` 가까운 픽셀끼리는 값이 비슷하다.
    * `ex2` 색상 RGB 채널끼리 밀접한 관련이 있다.
    * `ex3` 거리가 먼 픽셀들끼리는 관련이 없다.


반면, 합성곱 신경망 일명 CNN은 이 3차원 데이터를 입력 받아 3차원 형태로 다음 계층에 전달을 한다. 그렇게 하면서 이미지의 특징을 보존할 수 있다. 이러한 이미지와 같은 입력받고 출력하는 데이터를 CNN에서는 **feature map**이라고 한다.

## 3. 합성곱 계층
이 계층에서는 **이미지 데이터의 특징인 feature map**을 뽑는 과정이 진행된다.

### 3.1. 합성곱 연산 Convolution
합성곱 연산(Convolution)은 **필터 연산**이라 보면 된다. 필터를 씌우고 왼쪽에서 오른쪽, 위쪽에서 아래쪽으로 움직이면서 연산을 하는 것이다.

여기서의 연산은 **입력 데이터와 필터에서 대응하는 원소끼리 곱한 후에 그 합을 결과 데이터에 저장**한다.

<img src="./images/convolution.png" width=60%>

편향은 필터를 적용한 후에 더하면 된다. 편향은 항상 `1×1` 크기이다. 브로드캐스팅을 이용하여 필터가 된 데이터 요소 각각에 편향이 더해진다.

<img src="./images/convolution_bias.png" width=60%>

----

합성곱(Convolution)은 **이미지의 특징**을 골라낼 수 있도록 하는 연산이라고 보면 된다. 신경망 학습을 반복하면서 **이 필터의 값(= 매개변수)을 업데이트** 하면서 이미지의 특징을 잘 찾는 필터가 되도록 하는 것이다. 


$$ 
\text{Convolution filter} =
\begin{bmatrix} 
   -1 & 0 & 1 \\
   -1 & 0 & 1 \\
   -1 & 0 & 1
\end{bmatrix}
$$
<img src="./images/conv_mask1_image.JPG" width="600">

$$ 
\text{Convolution filter} =
\begin{bmatrix} 
   1 & 0 & -1 \\
   1 & 0 & -1 \\
   1 & 0 & -1
\end{bmatrix}
$$
<img src="./images/conv_mask2_image.JPG" width="600">


### 3.2. 패딩 Padding
패딩(Padding)은 **합성곱 연산으로 인해 원 데이터의 크기가 줄어드는 것을 방지**하기 위한 것이다. 합성곱 연산을 계속 하다가 결국에 출력 데이터의 크기가 1이 되어버리면 더이상 이 연산을 할 수 없다. 

그래서 우리가 추울 때 패딩을 입는 것처럼 특정 값(`ex. 0`)으로 원 데이터를 죽 둘러싸주는 것이다.

<img src="./images/padding.png" width=60%>

### 3.3. 스트라이드 Stride
스트라이드(Stride)는 **필터를 하는 윈도우를 몇 칸씩 이동할지**를 말한다. 예를 들어 스트라이드 2이면 윈도우가 두 칸씩 이동을 한다.

<img src="./images/stride.png" width="500">

### 3.4. 출력 데이터의 크기
아래의 식을 이용하면 출력 데이터의 크기를 구할 수 있다. 단, 값은 무조건 **정수로 나누어떨어져**야 한다.
* $(H, W)$ : (`입력 데이터`의 높이, `입력 데이터`의 너비)
* $(FH, FW)$ : (`필터`의 높이, `필터`의 너비)
* $(OH, OW)$ : (`출력 데이터`의 높이, `출력 데이터`의 너비)
* $P$ : 패딩
* $S$ : 스트라이드

$$OH = \frac{H+2P-FH}{S} + 1$$

$$OW = \frac{W+2P-FW}{S} + 1$$

### 3.5. 3차원 데이터의 합성곱 연산
3차원 데이터는 `(세로, 가로, 채널)`로 되어있는 데이터이다. 이런 3차원 데이터로 합성곱 연산을 할 때에는 **채널의 크기만큼 필터가 존재**해야 하고 채널마다 연산을 해 그 결과를 다 더해서 출력값을 얻는다.

복잡하게 생각하지 말고 **각 채널마다 맞추어진 필터가 있는데 각각 씌워서 연산 값을 다 합하는 거**라 생각하면 된다. 채널이 3개 있고 각각을 `RGB`를 나타내었다고 할 때 각 채널에 맞는 필터를 씌워 값을 구한다 생각하자.

<img src="./images/3d_fm_conv.png" width="500">

### 3.6. 블록으로 생각하기
위처럼 생각하지 말고 3차원 형상의 데이터, 필터를 **블록**으로 생각하면 쉽다. 이런 식으로 **구한 출력 데이터 하나가 이미지를 대표하는 특징인 feature map**이 되는 것이다.

#### 3.6.1. 필터가 1개인 경우
입력 데이터가 $1$개이고, 이 입력데이터에 대한 특징(=필터)도 $1$개

<img src="./images/fn_1.png" width=70%>

#### 3.6.2. 필터가 FN개인 경우
입력 데이터가 $1$개이고, 이 입력데이터에 대한 특징(=필터)은 $FN$개

<img src="./images/fn_FN.png" width=70%>

#### 3.6.3. 필터가 FN개, 입력 데이터가 N개인 경우
입력 데이터가 $N$개이고, 이 입력데이터에 대한 특징(=필터)은 $FN$개

<img src="./images/fn_FN_data_N.png" width=70%>

## 4. 풀링 계층
우리가 다루는 이미지 데이터는 생각 이상으로 매우 큰데다가 데이터의 숫자가 많다. 그래서 엄청난 양의 연산이 필요하고 그 연산으로 인한 시간과 비용이 상당하다. 그래서 이 풀링 계층을 통해 이미지의 특징에서도 **대표적인 것**들만 쏙쏙 뽑아내는 것이다. 
* `특징1` : 학습해야할 매개변수가 없다. 최대값 혹은 평균만 구하면 된다.
* `특징2` : 채널의 개수가 변하지 않는다. 계산만 해서 크기만 줄일 뿐 **특징의 개수**는 유지된다.
* `특징3` : 입력 데이터가 조금 변화해도 풀링의 결과는 크게 변하지 않는다.

----

보통 풀링의 윈도우에 있는 값 중 가장 큰 값을 결과 값으로 잡는 **최대 풀링**이 많이 쓰이며, `풀링의 윈도우 크기 == 스트라이드`여야 한다. 윈도우가 이동을 해도 앞의 데이터와 겹쳐지지 않게 하는 것이다. 즉, 해당 영역에서 대표적인 특징을 뽑기 위해서이다.

다음 예는 `스트라이드 = 2`이고 `풀링 윈도우 크기 = 2`일 때의 최대 풀링을 처리하는 과정이다.

<img src="./images/max_pooling.png" width=70%>

## 5. CNN 시각화

학습을 하면서 필터의 값 즉, 매개변수가 업데이트 되면서 **규칙성 있는, 패턴을 보이는 필터**로 바뀌었다.

<img src="./images/cnn_viz1.png" width=70%>

----

`필터1`은 세로 테두리를 인식하고, `필터2`는 가로 테두리를 인식한다. 이런 필터들(=특징들)을 모아서 똑같지만 모양은 조금 다른 이미지들을 분류할 수 있게 된다.

<img src="./images/cnn_viz2.png" width=70%>

----

층을 깊이하면 할수록 단순히 테투리만 인식하는 것에 떠나서 재질, 사물의 일부 등 **고급 정보**까지도 인식하게 된다. 즉, 사람처럼 사물의 의미를 이해하게 된다.

<img src="./images/cnn_viz3.png" width=70%>